In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env
import datetime

# Exercises
- file name: `time_series_anomaly_detection.py` or `time_series_anomaly_detection.ipynb`

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

**Hint**: You will need to explore the database (yay SQL!)

1. Label students by the program they are in.
1. Is it possible to identify the user_id of a staff member?
1. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.
1. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2. 
1. Plot the access activity of these students.

**BONUS:** Can you identify users who are viewing both the web dev and data science curriculum?

In [3]:
def acquire_cohort_logs(user=env.user, password=env.password, host=env.host):
    '''
    This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
    '''
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
    query = '''
    SELECT date, path as endpoint, user_id, cohort_id, name, start_date, end_date, ip as source_ip, program_id
        FROM logs l
        LEFT JOIN cohorts c ON l.cohort_id = c.id;
    '''
    df = pd.read_sql(query, url)
    return df

In [3]:
#df = acquire_cohort_logs(user=env.user, password=env.password, host=env.host)
#Save file to csv

# saving the dataframe
#df.to_csv('cohort_logs.csv')

In [4]:
#Get csv from file
df = pd.read_csv('cohort_logs.csv', index_col=[0])

In [5]:
df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id
0,2018-01-26,/,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0
1,2018-01-26,java-ii,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0
3,2018-01-26,slides/object_oriented_programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0
4,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0


### 1. Label students by the program they are in.

In [6]:
#Changing date, start and end dates to datetime fields
#Make a datetime object
df["date"]= pd.to_datetime(df["date"])
df["start_date"]= pd.to_datetime(df["start_date"])
df["end_date"]= pd.to_datetime(df["end_date"])

In [7]:
#df['program_length'] = df.start_date - df.end_date
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   endpoint    900222 non-null  object        
 2   user_id     900223 non-null  int64         
 3   cohort_id   847330 non-null  float64       
 4   name        847330 non-null  object        
 5   start_date  847330 non-null  datetime64[ns]
 6   end_date    847330 non-null  datetime64[ns]
 7   source_ip   900223 non-null  object        
 8   program_id  847330 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(1), object(3)
memory usage: 68.7+ MB


In [8]:
time_diff= df.end_date - df.start_date

In [9]:
df['program_length'] = df.end_date - df.start_date
df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length
0,2018-01-26,/,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days
1,2018-01-26,java-ii,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days
3,2018-01-26,slides/object_oriented_programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days
4,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   date            900223 non-null  datetime64[ns] 
 1   endpoint        900222 non-null  object         
 2   user_id         900223 non-null  int64          
 3   cohort_id       847330 non-null  float64        
 4   name            847330 non-null  object         
 5   start_date      847330 non-null  datetime64[ns] 
 6   end_date        847330 non-null  datetime64[ns] 
 7   source_ip       900223 non-null  object         
 8   program_id      847330 non-null  float64        
 9   program_length  847330 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), float64(2), int64(1), object(3), timedelta64[ns](1)
memory usage: 75.5+ MB


In [11]:
df.program_id.value_counts()

2.0    713365
3.0    103412
1.0     30548
4.0         5
Name: program_id, dtype: int64

In [17]:
#Create column with program name
#Change program id to object
df["program_id"]= df["program_id"].astype(str)
#Copying these values to a new column program name
df['program_name'] = df['program_id']
#Mapping the real names of the program
df["program_name"] = df["program_name"].map({'1.0':'full stack PHP','2.0':'full stack Java','3.0':'data science','4.0':'staff'})

In [18]:
df.head()

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
0,2018-01-26,/,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
1,2018-01-26,java-ii,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
3,2018-01-26,slides/object_oriented_programming,1,8.0,Hampton,2015-09-22,2016-02-06,97.105.19.61,1.0,137 days,full stack PHP
4,2018-01-26,javascript-i/conditionals,2,22.0,Teddy,2018-01-08,2018-05-17,97.105.19.61,2.0,129 days,full stack Java


### 2. Is it possible to identify the user_id of a staff member?

#### Yes, staff have program id 4


In [55]:
df[df.program_id == 4]

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length
31627,2018-03-22,/,152,9.0,Apollo,2015-03-30,2015-07-29,207.68.209.17,4.0,121 days
31628,2018-03-22,content/html-css,152,9.0,Apollo,2015-03-30,2015-07-29,207.68.209.17,4.0,121 days
31629,2018-03-22,content/html-css/gitbook/images/favicon.ico,152,9.0,Apollo,2015-03-30,2015-07-29,207.68.209.17,4.0,121 days
31630,2018-03-22,content/html-css,152,9.0,Apollo,2015-03-30,2015-07-29,207.68.209.17,4.0,121 days
31631,2018-03-22,content/html-css/introduction.html,152,9.0,Apollo,2015-03-30,2015-07-29,207.68.209.17,4.0,121 days


### 3. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [21]:
df[df.date > df.end_date].user_id.nunique()

592

In [27]:
after_codeup = df[df.date > df.end_date].groupby(df.user_id)

In [28]:
after_codeup